In [1]:
import numpy as np
from python.notebook_utils import look
from python.simulation_utils import simulation_wrapper, OpSim_wrapper

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import lsst.afw.display.rgb as rgb

Load the 2018 baseline survey

In [3]:
n_obs_list = [8, 8, 8]
repeat_obs = 3  # number of different fields with the given n_obs to generate simulations of
template_year = 1 
alert_year = 2
n_star = 10000
n_quasar = 1000

Connect to the database and set up for all of the simulations

Create a second set of simulations where the fields can be mosaiced together

In [4]:
airmass_threshold = 1.15  # Target fields must include at least one observation above this airmass to be included
opsim = OpSim_wrapper(year=template_year, filt='g', opsim_db='/Users/sullivan/LSST/OpSim/baseline2018a.db',
                      sim_directory="/Users/sullivan/LSST/simulations/OpSim/pontus_2568/",
                      conditions_db="/Users/sullivan/LSST/OpSim/pontus_2568.db",
                      airmass_threshold=airmass_threshold)

In [5]:
nx = repeat_obs
ny = len(n_obs_list)
n = nx if nx > ny else ny
simulation_size = n
dither_scale = 0.
dither_x = dither_scale*(np.arange(nx) - (nx - 1)/2)
dither_y = dither_scale*(np.arange(ny) - (ny - 1)/2)


In [6]:
initial_seeing_range = 1.
initial_var = True
seed = 42
randomize_conditions = True
randomize_template = True
n_alert = 24

Make a text file containing the visit list

In [7]:
visits = None
mosaic = True
randomize_conditions = True
band_dict = {'u': 0, 'g': 1, 'r': 2, 'i': 3, 'z': 4, 'y': 5}
seeing_dict = {}
airmass_dict = {}
rng = np.random.RandomState(seed+3)
index = int(np.rint(rng.rand()*1000))
field_Id = 100
alert_field_Id = 0
seeing_range = initial_seeing_range
for d_y, n_obs in zip(dither_y, n_obs_list):
    for rpt in range(repeat_obs):
        d_x = dither_x[rpt]
        opsim.set_field(n_obs=4, index=index, year=template_year)
        field_Id0 = opsim.field_Id
        if randomize_template:
            opsim.update_year(template_year, randomize_conditions=randomize_conditions,
                              set_n_obs=n_obs, force_seeing_range=seeing_range)
        n_obs1 = len(opsim.airmass)
        for t_i in range(n_obs1):
            fieldId_i = 100*field_Id + 1000000*template_year + t_i
            seeing_dict[fieldId_i] = opsim.seeing[t_i]
            airmass_dict[fieldId_i] = opsim.airmass[t_i]
        
        opsim.field_Id = field_Id
        if mosaic:
            visit_name1 = "%i" % (100*opsim.field_Id + 1000000*template_year)
            visit_name1b = "%i" % (100*opsim.field_Id + 1000000*template_year + n_obs1 - 1)
        else:
            visit_name1 = "%i" % (100*band_dict[opsim.filter] + 1000*template_year)
            visit_name1b = "%i" % (100*band_dict[opsim.filter] + 1000*template_year + n_obs1 - 1)
        if visits is None:
            visits = visit_name1
        else:
            visits += "^" + visit_name1
        visits += "..%s" % visit_name1b
        field_Id += 1
        seeing_range *= 1.1

d_x = 0
d_y = 0
opsim.set_field(n_obs=4, index=index, year=alert_year)
if randomize_template:
    opsim.update_year(alert_year, randomize_conditions=randomize_conditions,
                      set_n_obs=n_alert)
n_obs1 = len(opsim.airmass)
for t_i in range(n_obs1):
    fieldId_i = 100*alert_field_Id + 1000000*alert_year + t_i
    seeing_dict[fieldId_i] = opsim.seeing[t_i]
    airmass_dict[fieldId_i] = opsim.airmass[t_i]
opsim.field_Id = alert_field_Id
if mosaic:
    visit_name2 = "%i" % (100*opsim.field_Id + 1000000*alert_year)
    visit_name2b = "%i" % (100*opsim.field_Id + 1000000*alert_year + n_obs1 - 1)
else:
    visit_name2 = "%i" % (100*band_dict[opsim.filter] + 1000*alert_year)
    visit_name2b = "%i" % (100*band_dict[opsim.filter] + 1000*alert_year + n_obs1 - 1)
visits += "^" + visit_name2
visits += "..%s" % visit_name2b

Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Selecting 8 randomized obs from field 2575, with seeing range 0.524 to 0.524 and airmass range 1.012 to 1.484
Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Selecting 8 randomized obs from field 2575, with seeing range 0.524 to 0.577 and airmass range 1.012 to 1.484
Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Selecting 8 randomized obs from field 2575, with seeing range 0.524 to 0.635 and airmass range 1.012 to 1.484
Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Selecting 8 randomized obs from field 2575, with seeing range 0.524 to 0.698 and airmass range 1.012 to 1.484
Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Selecting 8 randomized obs from field 2575, with seeing rang

In [9]:
seeing_dict

{1010000: 0.5244374092174477,
 1010001: 0.5244374092174477,
 1010002: 0.5244374092174477,
 1010003: 0.5244374092174477,
 1010004: 0.5244374092174477,
 1010005: 0.5244374092174477,
 1010006: 0.5244374092174477,
 1010007: 0.5244374092174477,
 1010100: 0.5768811501391926,
 1010101: 0.5372874608334048,
 1010102: 0.5583353681225943,
 1010103: 0.5431252897328596,
 1010104: 0.5350322774828323,
 1010105: 0.566082489925671,
 1010106: 0.5244374092174477,
 1010107: 0.5577056444149199,
 1010200: 0.6345692651531118,
 1010201: 0.5514225176109577,
 1010202: 0.5956231229182556,
 1010203: 0.5636819582998125,
 1010204: 0.5466866325747554,
 1010205: 0.6118920787047166,
 1010206: 0.5244374092174477,
 1010207: 0.5943007031321392,
 1010300: 0.6980261916684232,
 1010301: 0.5669710800662658,
 1010302: 0.636639653193483,
 1010303: 0.5862942937234608,
 1010304: 0.5595064231758706,
 1010305: 0.6622826263616668,
 1010306: 0.5244374092174477,
 1010307: 0.6345552677210806,
 1010400: 0.7678288108352656,
 1010401: 0.

In [80]:
print(visits)

1010000..1010007^1010100..1010107^1010200..1010207^1010300..1010307^1010400..1010407^1010500..1010507^1010600..1010607^1010700..1010707^1010800..1010807^2000000..2000023


In [81]:
output_directory = opsim.sim_directory + "mosaic_%i_varPSF_rand/" % seed
visits_file = output_directory + "visits.txt"
visits_cmd = "--id visit=" + visits
np.savetxt(visits_file, [visits_cmd], fmt='%s')

In [82]:
visits_cmd

'--id visit=1010000..1010007^1010100..1010107^1010200..1010207^1010300..1010307^1010400..1010407^1010500..1010507^1010600..1010607^1010700..1010707^1010800..1010807^2000000..2000023'

Run the actual simulations and write the fits files

In [77]:
rng = np.random.RandomState(seed+3)
index = int(np.rint(rng.rand()*1000))
field_Id = 100
alert_field_Id = 0
seeing_range = initial_seeing_range
for d_y, n_obs in zip(dither_y, n_obs_list):
    for rpt in range(repeat_obs):
        d_x = dither_x[rpt]
        opsim.set_field(n_obs=4, index=index, year=template_year)
        field_Id1 = opsim.field_Id
        sim = opsim.initialize_simulation(n_star=n_star*n**2, n_quasar=n_quasar*n**2,
                                          dither=(d_x, d_y), seed=seed,
                                          simulation_size=simulation_size)
        if randomize_template:
            opsim.update_year(template_year, randomize_conditions=randomize_conditions,
                              set_n_obs=n_obs, force_seeing_range=seeing_range)
        opsim.field_Id = field_Id
        opsim.run_simulation(sim, use_seeing=True, write_catalog=initial_var,
                             initialize_directory=initial_var, mosaic=True,
                             randomize_conditions=randomize_conditions)
        initial_var = False
        opsim.field_Id = field_Id1
        field_Id += 1
        seeing_range *= 1.1

d_x = 0
d_y = 0
opsim.set_field(n_obs=4, index=index, year=alert_year)
if randomize_template:
    opsim.update_year(alert_year, randomize_conditions=randomize_conditions,
                      set_n_obs=n_alert)
opsim.field_Id = alert_field_Id
opsim.run_simulation(sim, use_seeing=True, write_catalog=False,
                     initialize_directory=False, mosaic=True,
                     randomize_conditions=randomize_conditions)


Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Number and flux contribution of stars of each type:
 [M 68881| 1.79%] [K 10991| 2.09%] [G 6860| 3.94%] [F 2624| 4.82%] [A 537| 17.45%] [B 107| 69.90%] [O 0| 0.00%]
Simulating 2509 stars within observable region
Time to model 2502 stars: [13.296s | 0.00531s per star]
Time to model 7 bright stars: [9.387s | 1.34100s per star]
Simulating 252 quasars within observable region
Time to model 250 stars: [4.091s | 0.01636s per star]
Time to model 2 bright stars: [12.102s | 6.05085s per star]
Selecting 8 randomized obs from field 2575, with seeing range 0.524 to 0.524 and airmass range 1.012 to 1.484
Writing 6217 stars brighter than 16.0 mag to reference catalog in 1 bands
Min/max magnitude:  6.086482643174753 22.662486582014743
FFT timing for 15 DCR planes: [3.585s | 0.239s per plane]
FFT timing for 15 DCR planes: [12.444s | 0.830s per plane]
FFT timing for 15 DCR planes: [1.838s | 0.123s per pla

FFT timing for 15 DCR planes: [6.868s | 0.458s per plane]
Selecting 13 obs from field 2575, with seeing range 0.622 to 0.999 and airmass range 1.123 to 1.190
Number and flux contribution of stars of each type:
 [M 68881| 1.79%] [K 10991| 2.09%] [G 6860| 3.94%] [F 2624| 4.82%] [A 537| 17.45%] [B 107| 69.90%] [O 0| 0.00%]
Simulating 2509 stars within observable region
Time to model 2502 stars: [11.893s | 0.00475s per star]
Time to model 7 bright stars: [8.050s | 1.15007s per star]
Simulating 252 quasars within observable region
Time to model 250 stars: [3.375s | 0.01350s per star]
Time to model 2 bright stars: [9.904s | 4.95206s per star]
Selecting 8 randomized obs from field 2575, with seeing range 0.524 to 0.845 and airmass range 1.012 to 1.484
FFT timing for 15 DCR planes: [4.209s | 0.281s per plane]
FFT timing for 15 DCR planes: [7.391s | 0.493s per plane]
FFT timing for 15 DCR planes: [1.850s | 0.123s per plane]
FFT timing for 15 DCR planes: [6.859s | 0.457s per plane]
FFT timing fo

FFT timing for 15 DCR planes: [6.693s | 0.446s per plane]
FFT timing for 15 DCR planes: [1.845s | 0.123s per plane]
FFT timing for 15 DCR planes: [6.758s | 0.451s per plane]
FFT timing for 15 DCR planes: [1.638s | 0.109s per plane]
FFT timing for 15 DCR planes: [6.691s | 0.446s per plane]
FFT timing for 15 DCR planes: [1.991s | 0.133s per plane]
FFT timing for 15 DCR planes: [6.701s | 0.447s per plane]
FFT timing for 15 DCR planes: [1.749s | 0.117s per plane]
FFT timing for 15 DCR planes: [6.718s | 0.448s per plane]
FFT timing for 15 DCR planes: [1.632s | 0.109s per plane]
FFT timing for 15 DCR planes: [6.717s | 0.448s per plane]
FFT timing for 15 DCR planes: [1.879s | 0.125s per plane]
FFT timing for 15 DCR planes: [6.764s | 0.451s per plane]
FFT timing for 15 DCR planes: [2.028s | 0.135s per plane]
FFT timing for 15 DCR planes: [6.744s | 0.450s per plane]
FFT timing for 15 DCR planes: [1.723s | 0.115s per plane]
FFT timing for 15 DCR planes: [6.709s | 0.447s per plane]
FFT timing for